## 4. <a id='4_cell'></a>Create Group using existing Da Vinci Alerts Data (from Synthea) and the PyFHIR models to...

- create Group resources for discovery by:

    1. all patients
    1. all patients with managing org
    1. practitioner characteristic
    1. location characteristic    
    *TODO*
    - *add narrative*
    - *add Careteams and Careteam characteristics*
- Create the Q/QR data using existing tools and add extensions to resources
  - [X]  update Q and QR data
  - [X]  move extension to member 
  - [X] confirm url
- [X] Add extensions for Encounter and Appt to resource
    - [X] update Encounter ID - with mapping
    - [X] create Appt resource and use
    
- Create transaction bundle of resources
- Load to ref server
- Fetching and...
- Getting Additional data by:
    1. FHIR RESTful queries on the:
       - Patient
       - *TODO Condition and Encounter resources
      

### Create Group Instance for All Patients, One member for Each Row...

1. initialize group resource and datatype instances
1. iterate over each Patient in Bundle
1. validate and save the resources in the reference server
    tag them so can search more easily

TODO
1. option to create narrative using jinja2 templates (that is a topic for another notebook)
1. create a new transaction Bundle with all the data (figure out sequence for encounters)

### import all the modules you need

In [1]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# This allows you to import the desired function from the module hierarchy:
from pyFHIR_models.fhir_model_generator.model import (
                                            fhirdate,
                                            fhirreference,
                                            group,
                                            patient,
                                            organization,
                                            bundle,
                                            identifier,
                                            humanname,
                                            contactpoint,
                                            address,
                                            codeableconcept,
                                            coding,
                                            extension,
                                            questionnaire as q,
                                            questionnaireresponse as qr,
                                            appointment as a,
                                             )
from json import loads, dumps
#from requests import get, post, put
from datetime import datetime, date, timedelta, timezone
from IPython.display import display as Display, HTML, Markdown
from utils import bundle_me, to_json, to_yaml, get_id, load_transaction, validate, fetch_r
from pathlib import Path
#from pprint import pprint, pformat
from nested_lookup import nested_lookup
import FHIR_templates
from random import randrange,choice

## Globals

file location etc...

'pl_use_case':

|value|what it does|
|---|---|
|all| create Group with no characterisitics|
|location| create Group with location characteristic|
|practitioner| create Group with practitioner characteristic|


In [2]:
"""
group_characterstics = [
(0,'location','Location/[id]',),
(1,'practitioner','Practitioner/[id]',),
(2,'organization','Organization/[id]',),
(3,'team','CareTeam/[id]',),
]
"""

base_path = '/Users/ehaas/Documents/FHIR/Davinci-Alerts/2020_09_hl7_connectathon/Synthea_Alert_Test_Data'

file_type = 'fhir'
#file_type = 'csv'

file_size = '3Patients'
#file_size = '100Patients'

file = 'admit_notify-2.json'
#file = 'admit_notify-100.json' # don't use - too many ManagingEntities
#file = 'admit_notify-100r1.json' # use me instead
#file = 'admit_notify-100-BEVERLY HOSPITAL CORPORATION.json'
#file = 'admit_notify-100-HALLMARK HEALTH SYSTEM.json'
#file = 'admit_notify-100-CAMBRIDGE HEALTH ALLIANCE.json'


mypath = Path() / base_path / file_type / file_size
my_infile = mypath/file

print(f'My Path to Synthea Data a FHIR Bundle is {my_infile}')
print()

console_only = False

chars = dict(  # this is dict to provide the FHIR path to the reference in Encounters to get data
#all = None,
location = 'location.location',
practitioner = 'participant.individual',
)

example_series = dict(
all = 100,
location = 200,
practitioner = 300,
)

#pl_use_case = "location"
#pl_use_case = "practitioner"
pl_use_case = "all"
my_tags = [{'code': '2020-Sep'},{'code': '2021-Jan'}]
group_profile = 'http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist'
char_system = 'http://www.fhir.org/guides/argonaut/patient-list/CodeSystem/group-characteristic'
q_ext_url = 'http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-questionnaire'
QuestionnaireResponse_ext_url = 'http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-questionnaireresponse'
Appointment_ext_url = 'http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-appointment'
enc_ext_url = 'http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-encounter'
q_id = 'argo-pl-q2'


display(HTML(f'<h1>USE CASE = {pl_use_case}</h1>'))

My Path to Synthea Data a FHIR Bundle is /Users/ehaas/Documents/FHIR/Davinci-Alerts/2020_09_hl7_connectathon/Synthea_Alert_Test_Data/fhir/3Patients/admit_notify-2.json



## Functions

In [3]:
def get_my_chars():  #  get reference based on dot notation string
    my_chars = []
    print(f'get ref from Encounter.{chars[pl_use_case]}')
    for i in my_bundle.entry:
        r = i.resource
        if r.resource_type == "Encounter" and get_id(r.serviceProvider) == get_id(me):
            #print(f'r.serviceProvider.reference == {get_id(r.serviceProvider)} /n me.reference = {get_id(me)}')
            e = r
            
            for j in chars[pl_use_case].split("."):  # creat a util for this nested getattr
                #print(j)
                try:
                    e = getattr(e,j)
                except AttributeError:  # maybe a list so try getting the first element of list
                    e = getattr(e[0],j)
            #print(e)
            my_chars.append(e)
    #print('my-chars', len(my_chars))
    my_chars = dedup(my_chars)
    #print('my-chars deduped',len(my_chars))
    return my_chars
              
    
def add_char(): #Add A Single Group Characteristic
    char.reference = f'{pl_use_case.capitalize()}/{get_id(char)}'
    #print(to_yaml(me))
    my_g.characteristic = []
    my_char = group.GroupCharacteristic()
    my_char.code = codeableconcept.CodeableConcept()
    my_char.code.coding = [coding.Coding()]
    my_char.code.coding[0].code = pl_use_case
    my_char.code.coding[0].display = pl_use_case.capitalize()
    my_char.code.coding[0].system = char_system
    my_char.valueReference = char
    my_char.exclude = False
    my_g.characteristic.append(my_char)
    
    

def dedup(my_refs):
    deduped = [fhirreference.FHIRReference(dict(t)) for t in {tuple(d.as_json().items()) for d in my_refs}] #dedup
    return deduped
    
    

def add_qr():  # Add the QR
    pass
    
def counter():
     if file_size == '3Patients':
        return len(my_groups) + example_series[pl_use_case]
     else:
        return len(my_groups) + example_series[pl_use_case]*10
    
def create_new_group(counter):
    my_id = f'argo-pl-group-{pl_use_case}-{counter}'
    my_meta = dict(
        source = "Health_eData_Inc",
        tag = my_tags,
        lastUpdated= f'{datetime.now().isoformat()}Z',
        profile = [group_profile,],
        )

    my_identifier = dict(
            value = my_id,
            system = "http://example.org/fhir/identifiers",
            )
    
    q_ref = dict(
    reference = f'Questionnaire/{q_id}',
    display = f'Argo PL List Test Questionnaire id={q_id}',
     )
    
    my_q_extension = dict(
     url = q_ext_url,
     valueReference = q_ref,
    )

    my_g = group.Group(
            dict(
                meta = my_meta,
                id = my_id,
                extension = [my_q_extension],
                type = 'person',
                actual = True,
                name = f'Argo Patient List #{counter}: "{pl_use_case}" Use Case',
                active = True,
                identifier = [my_identifier],
                #quantity = df.shape[0],         
            )
    ) # initialize group instance
    
    # Add the managingEntity
    me.reference = f'Organization/{get_id(me)}'
    #print(to_yaml(me))
    my_g.managingEntity = me

    #print(to_yaml(my_g))
    return my_g


### Add managingEntity element, A Single Group Characteristic and Q extensions

Fetch the FHIR Bundle of Synthea Data -
- path variables are local for this - change as needed

#### add the managingEntity element - i.e. the Organization that created the group.
 - note there are no managing Org on these patients. so for this data set we impute the organization from the the encounter to tell us which organization the patient list go with.
   (This is an issue that is system dependent only they know)
- use nested lookup to extract the Me from encounter
- use FHIR references for now ( may need to switch back to urns for bundling)

#### Add A Single Group Characteristic

- based on Encounter add Location and Practitioner to start 
- TODO add Organization and CareTeam when define these resources ( what does Organization mean?)
- create a function and run through the parameters to build the element

#### Add A Q Extensions

- Add extension to all Groups first
- Create and use fixed templates Q's from NIH Tool
- See if tooling can autopopulate QRs otherwise use my own base on FHIRpath

#### Add An Appt Extensions

- Add extension to all Groups first
- Create Appt based on template

#### Add An Encounter Extensions

- Add extension to all Groups first
- Map Encounter to Patient (or based on csv)
- Create Encounter from mapping

In [4]:
'''
json_bundles = []
for f in mypath.iterdir():
    #print(type(f), f)
    json_bundles.append(f.read_text(encoding='utf-8'))
'''

my_groups = []
my_bundle = bundle.Bundle(loads(my_infile.read_text(encoding='utf-8')))
print(f'len(my_bundle.entry)={len(my_bundle.entry)}' )
my_refs = [r.resource.serviceProvider for r in my_bundle.entry if r.resource.resource_type == "Encounter"]
my_refs = dedup(my_refs)
for i in my_refs:
    display(HTML('<h1>MY_REFS AS YAML</h1>'))
    display(HTML(f'<pre style="color: blue; background-color: AliceBlue;">{to_yaml(i)}</pre>'))

len(my_bundle.entry)=21


In [5]:
my_groups = []

for me in my_refs:
    
    if pl_use_case in chars.keys():
        display(HTML(f'<h3>Use Case = {pl_use_case} for {me.display}</h3>'))
        my_chars = get_my_chars() # get all chars for me 
        #print(my_chars)
        for char in my_chars:
            my_g = create_new_group(counter())
            add_char()
            my_groups.append(my_g)

    else: 

        my_g = create_new_group(counter())

        my_groups.append(my_g)

    
my_members = {k.id:set() for k in my_groups} # to index members and prevent dupes

for i in my_groups:
    display(HTML('<h1>GROUP RESOURCE AS YAML</h1>'))
    display(HTML(f'<pre style="color: blue; background-color: AliceBlue;">{to_yaml(i)}</pre>'))
print(f'len(my_bundle.entry)={len([x for x in my_bundle.entry if x.resource.resource_type == "Encounter"])}' )
print(f'len(my_groups)={len(my_groups)}' )

len(my_bundle.entry)=3
len(my_groups)=3


### Add the Group Members from Bundle for Each ManagingEntity

- Fetch Bundle from local dir
- get Patient Ids for each organization
  - note there are no managing Org on these patients. so for this data set we impute the organization from the the encounter to tell us which organization the patient list go with.
   (This is an issue that is system dependent only they know)
   cycle through all encounters to get all patients for each ME
- Create QR for each Member based on Q and add extension
    Use template for now
    - [X] update QR to normalized questions
- [X] Create Appt for each Member based and add Appt extension
    Use template for now
    - [X] add patient ID
    - [X] update Encounter to reference Appt
    - [X] update organization and Practitioner based on Encounter
- [X] Get Encounter ID for each Member based on and add Enc extension
- [X] add quantity element


In [6]:
def make_QuestionnaireResponse():
    '''
    make  QR resource based on type and return resource id and bundle up QRs 
    '''
    p_id = get_id(pyfhir.subject)
    my_qr = qr.QuestionnaireResponse(FHIR_templates.qr_template) # instantiate QR from premade template
    my_qr.id = f'{q_id}-qr-{p_id}'
    if pl_use_case == "all":  # only need to do to create the references resources once this once ASSUMES START WITH 'ALL'
        my_qr.subject.reference = f'Patient/{p_id}'
        my_qr.subject.display = pyfhir.subject.display
        now = f'{datetime.now().isoformat()}Z'
        my_qr.authored = fhirdate.FHIRDate(now)
        #get source data using FHIR RESTful Searches for items 0 and 1
        # *********************** QR RESPONSES ************************
        r_dict = fetch_r('Patient',p_id) #get p as dict to answer sham questions
        py_patient = patient.Patient(r_dict)                                                      
        my_qr.item[2].answer[0].valueInteger = randrange(13,45,1)
        my_qr.item[3].answer[0].valueInteger = randrange(-10,10,1)
        my_qr.item[4].answer[0].valueInteger = randrange(1,9,1)
        my_qr.item[1].answer[0].valueString =  f'{randrange(20,90,1)}/{choice(["Male","Female"])}'
        #print(to_yaml(my_qr))
        my_qrs.append(my_qr)
    return my_qr.id
    
    
def make_Appointment():
    '''
    make resource Appts resource and return resource id and bundle up Appts 
    '''
    p_id = get_id(pyfhir.subject)
    my_appt = a.Appointment(FHIR_templates.appt_template) # instantiate Appt from premade template
    my_appt.id  =  f'{my_appt.id}-{p_id}'
    if pl_use_case == "all":  # only need to do to create the references resources once this once ASSUMES START WITH ALL
        pyfhir.appointment = [fhirreference.FHIRReference({'reference':f'Appointment/{my_appt.id}'})]  #assign appt to encounter too and update in transaction
        my_encs.append(pyfhir)
        my_appt.participant[0].actor.reference = f'Patient/{p_id}'
        my_appt.participant[0].actor.display = pyfhir.subject.display
        my_appt.participant[1].actor.reference = pyfhir.participant[0].individual.reference
        my_appt.participant[1].actor.display = pyfhir.participant[0].individual.display
        my_appt.participant[2].actor.reference = pyfhir.location[0].location.reference
        my_appt.participant[2].actor.display = pyfhir.location[0].location.display
        #print(to_yaml(my_appt))
        now = datetime.now(timezone.utc)
        then = now + timedelta(minutes=30)
        my_appt.start = fhirdate.FHIRDate(now.isoformat())
        my_appt.end = fhirdate.FHIRDate(then.isoformat())
        my_appts.append(my_appt)
    return my_appt.id

def get_my_ext(r_type, **kwargs):
    p_id = get_id(pyfhir.subject)
    
    my_id = globals()[f'make_{r_type}']() #  eg make_qr() make resource based on type and return resource id
    
    my_ref = dict(
    reference = f'{r_type}/{my_id}',
    display = f'Argo PL List Test {r_type} {my_id}',
     )
    my_url = globals()[f'{r_type}_ext_url']
    #print(my_url)
    my_extension = dict(
     url = my_url,
     valueReference = my_ref,
    )
    #print(to_yaml(my_extension))
    return my_extension
    
my_qrs = []
my_appts = []
my_encs = []
for count, i in enumerate(my_bundle.entry):
    print(count, i.resource.resource_type)
    my_chars = []
    pyfhir = i.resource
    if pyfhir.resource_type == "Encounter":
        #print(count,':',pyfhir.resource_type ,i.fullUrl, i.request ) 
        sp_id = get_id(pyfhir.serviceProvider)
        
        my_chars.append(get_id(pyfhir.participant[0].individual))
      
        my_chars.append(get_id(pyfhir.location[0].location))
        #print(count, my_chars)
        my_enc_ext = dict(
                     url = enc_ext_url,
                     valueReference = {'reference': f'Encounter/{pyfhir.id}', 
                                       'display': f'Argo PL List Test Encounter {pyfhir.id}',
                                      },
                        )
        my_qr_ext = get_my_ext(r_type="QuestionnaireResponse")
        #print(my_qr_ext)
        my_appt_ext = get_my_ext(r_type="Appointment")
        my_su = dict(
                reference = f"Patient/{get_id(pyfhir.subject)}",
                display = f"{pyfhir.subject.display}",
                )
        my_member = group.GroupMember(
            dict(
                extension = [my_appt_ext,my_enc_ext,my_qr_ext,],
                entity = my_su,
                period = {'start': str(date.today())},
                inactive = False,
                )
            )
        #print(f'my_member = {dumps(my_member.as_json(), indent=4)}')
        # iterate through the groups an append to Group with ME == ServiceProvider
        # and characteristice = characteristic  maintaining a set for each group
        for g in my_groups: # assuming a single characteristic for now..
            me_id =  get_id(g.managingEntity)
            #print(g.characteristic[0].valueReference)
            if pl_use_case == "all" or get_id(g.characteristic[0].valueReference) in my_chars:
                #print(f'my_member.entity.reference ={my_member.entity.reference}')
                if me_id == sp_id and my_member.entity.reference not in my_members[g.id]:
                    try:
                        g.member.append(my_member)
                    except AttributeError:
                        g.member=[my_member]
                    my_members[g.id].add(my_member.entity.reference)
                    #print(f'my_members = {my_members}')
            g.quantity = len(my_members[g.id])


    
for i in my_qrs:
    display(HTML('<h1>QR RESOURCE AS YAML</h1>'))
    display(HTML(f'<pre style="color: blue; background-color: AliceBlue;">{to_yaml(i)}</pre>'))
    display(HTML('...validating...&#9203;'))
    r = validate(i)
    display(HTML(f'<h2>Validation output</h1><h2>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}<hr><hr>'))
            
for i in my_appts:
    display(HTML('<h1>APPT RESOURCE AS YAML</h1>'))
    display(HTML(f'<pre style="color: blue; background-color: AliceBlue;">{to_yaml(i)}</pre>'))
    display(HTML('...validating...&#9203;'))
    r = validate(i)
    display(HTML(f'<h2>Validation output</h1><h2>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}<hr><hr>'))
    
for i in my_encs:
    display(HTML('<h1>ENCOUNTER RESOURCE AS YAML</h1>'))
    display(HTML(f'<pre style="color: blue; background-color: AliceBlue;">{to_yaml(i)}</pre>'))
    display(HTML('...validating...&#9203;'))
    r = validate(i)
    display(HTML(f'<h2>Validation output</h1><h2>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}<hr><hr>'))

for i in my_groups:
    display(HTML('<h1>GROUP RESOURCE AS YAML</h1>'))
    display(HTML(f'<h3>Quantity = {i.quantity}</h3>'))
    display(HTML(f'<pre style="color: blue; background-color: AliceBlue;">{to_yaml(i)}</pre>'))
    display(HTML('...validating...&#9203;'))
    r = validate(i)
    display(HTML(f'<h2>Validation output</h1><h2>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]} <hr><hr>'))


0 Patient
1 Coverage
2 Organization
3 Location
4 Organization
5 Practitioner
6 Encounter


7 Patient
8 Coverage
9 Organization
10 Location
11 Organization
12 Practitioner
13 Encounter


14 Patient
15 Coverage
16 Organization
17 Location
18 Organization
19 Practitioner
20 Encounter


WARNING,"[QuestionnaireResponse, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[QuestionnaireResponse.meta.tag[0], Line 9, Col 8]",A code with no system has no defined meaning. A system should be provided
WARNING,"[QuestionnaireResponse.meta.tag[1], Line 12, Col 8]",A code with no system has no defined meaning. A system should be provided
WARNING,"[QuestionnaireResponse, Line 17, Col 96]","The questionnaire 'http://www.fhir.org/guides/argonaut/patient-list/Questionniare/argo-pl-q2' could not be resolved, so no validation can be performed against the base questionnaire"


WARNING,"[QuestionnaireResponse, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[QuestionnaireResponse.meta.tag[0], Line 9, Col 8]",A code with no system has no defined meaning. A system should be provided
WARNING,"[QuestionnaireResponse.meta.tag[1], Line 12, Col 8]",A code with no system has no defined meaning. A system should be provided
WARNING,"[QuestionnaireResponse, Line 17, Col 96]","The questionnaire 'http://www.fhir.org/guides/argonaut/patient-list/Questionniare/argo-pl-q2' could not be resolved, so no validation can be performed against the base questionnaire"


...

WARNING,"[Encounter, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Encounter.type[0], Line 21, Col 6]","None of the codes provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-encounter-type (http://hl7.org/fhir/us/core/ValueSet/us-core-encounter-type), and a code should come from this value set unless it has no suitable code and the validator cannot judge what is suitable) (codes = http://example.org/fhir/conditions#261665006)"


WARNING,"[Encounter, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Encounter.type[0], Line 21, Col 6]","None of the codes provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-encounter-type (http://hl7.org/fhir/us/core/ValueSet/us-core-encounter-type), and a code should come from this value set unless it has no suitable code and the validator cannot judge what is suitable) (codes = http://example.org/fhir/conditions#261665006)"


WARNING,"[Group, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Group.meta.tag[0], Line 11, Col 8]",A code with no system has no defined meaning. A system should be provided
WARNING,"[Group.meta.tag[1], Line 14, Col 8]",A code with no system has no defined meaning. A system should be provided
INFORMATION,"[Group.extension[0], Line 20, Col 6]",Unknown extension http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-questionnaire
ERROR,"[Group.extension[0].value.ofType(Reference), Line 23, Col 49]",Unable to resolve resource 'Questionnaire/argo-pl-q2'
INFORMATION,"[Group.member[0].extension[0], Line 46, Col 10]",Unknown extension http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-appointment
INFORMATION,"[Group.member[0].extension[1], Line 53, Col 10]",Unknown extension http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-encounter
INFORMATION,"[Group.member[0].extension[2], Line 60, Col 10]",Unknown extension http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-questionnaireresponse
INFORMATION,"[Group.member[0].entity, Line 69, Col 69]",Details for Patient/b1cf5f57-b061-4b7f-aa9d-6283a121694b matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
ERROR,"[Group.meta.profile[0], Line 1, Col 2]","Profile reference 'http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist' could not be resolved, so has not been checked"


WARNING,"[Group, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Group.meta.tag[0], Line 11, Col 8]",A code with no system has no defined meaning. A system should be provided
WARNING,"[Group.meta.tag[1], Line 14, Col 8]",A code with no system has no defined meaning. A system should be provided
INFORMATION,"[Group.extension[0], Line 20, Col 6]",Unknown extension http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-questionnaire
ERROR,"[Group.extension[0].value.ofType(Reference), Line 23, Col 49]",Unable to resolve resource 'Questionnaire/argo-pl-q2'
INFORMATION,"[Group.member[0].extension[0], Line 46, Col 10]",Unknown extension http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-appointment
INFORMATION,"[Group.member[0].extension[1], Line 53, Col 10]",Unknown extension http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-encounter
INFORMATION,"[Group.member[0].extension[2], Line 60, Col 10]",Unknown extension http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-questionnaireresponse
INFORMATION,"[Group.member[0].entity, Line 69, Col 69]",Details for Patient/06e1f0dd-5fbe-4480-9bb4-6b54ec02d31b matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
ERROR,"[Group.meta.profile[0], Line 1, Col 2]","Profile reference 'http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist' could not be resolved, so has not been checked"


WARNING,"[Group, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Group.meta.tag[0], Line 11, Col 8]",A code with no system has no defined meaning. A system should be provided
WARNING,"[Group.meta.tag[1], Line 14, Col 8]",A code with no system has no defined meaning. A system should be provided
INFORMATION,"[Group.extension[0], Line 20, Col 6]",Unknown extension http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-questionnaire
ERROR,"[Group.extension[0].value.ofType(Reference), Line 23, Col 49]",Unable to resolve resource 'Questionnaire/argo-pl-q2'
INFORMATION,"[Group.member[0].extension[0], Line 46, Col 10]",Unknown extension http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-appointment
INFORMATION,"[Group.member[0].extension[1], Line 53, Col 10]",Unknown extension http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-encounter
INFORMATION,"[Group.member[0].extension[2], Line 60, Col 10]",Unknown extension http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist-questionnaireresponse
INFORMATION,"[Group.member[0].entity, Line 69, Col 69]",Details for Patient/aad0894e-47f4-4ffc-8fab-8fe5487110d2 matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
ERROR,"[Group.meta.profile[0], Line 1, Col 2]","Profile reference 'http://www.fhir.org/guides/argonaut/patient-list/StructureDefinition/patientlist' could not be resolved, so has not been checked"


### Rebundle into Transaction/Validate/Save/Load to ref FHIR Server

- Add new and updated resources to Ttansaction bundle using PUT to preserve ids 
  order is Patient, Pract, Org, Location, Careteam, Condition, Observation, Appointment, Encounter, Q, QR, Group
  
1. use original Bundle
2. remove old encounter Encounter
3. append Appt
4. append new Enc
5. append Q, QR, Group

In [7]:

display(HTML('......Bundle into transaction...... &#9203;'))
test_server = 'http://test.fhir.org/r4'

#remove old encounter Encounters
my_enc_ids = [i.id for i in my_encs]
print(my_enc_ids)
my_bundle.entry = [e for e in my_bundle.entry if e.resource.id not in my_enc_ids]
print([e.resource.resource_type for e in my_bundle.entry])

my_res =  my_appts + my_encs + [q.Questionnaire(FHIR_templates.my_q)] + my_qrs +  my_groups
for r in my_res:
    print(f'type = {r.resource_type}, id = {r.id}')
    try:
        my_b = bundle_me(test_server, r, my_bundle)
    except NameError:
        my_b = bundle_me(test_server, r)

display(HTML('<h1>BUNDLE RESOURCE AS YAML</h1>'))
display(HTML(f'<pre style="color: blue; background-color: AliceBlue;">{to_yaml(my_b)}</pre>'))
display(HTML('...validating...&#9203;'))
r = validate(my_b)
display(HTML(f'<h2>Validation output</h1><h2>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]} <hr><hr>'))

['5fe62cd5-bfcf-4d3b-a1e9-80d6f75d6f82', '542f9e32-4309-4277-81ce-12419f0d1294', '02ba9ec6-0712-4715-8ba4-5485fc571403']
['Patient', 'Coverage', 'Organization', 'Location', 'Organization', 'Practitioner', 'Patient', 'Coverage', 'Organization', 'Location', 'Organization', 'Practitioner', 'Patient', 'Coverage', 'Organization', 'Location', 'Organization', 'Practitioner']
type = Appointment, id = pl-appt-06e1f0dd-5fbe-4480-9bb4-6b54ec02d31b
type = Appointment, id = pl-appt-b1cf5f57-b061-4b7f-aa9d-6283a121694b
type = Appointment, id = pl-appt-aad0894e-47f4-4ffc-8fab-8fe5487110d2
type = Encounter, id = 5fe62cd5-bfcf-4d3b-a1e9-80d6f75d6f82
type = Encounter, id = 542f9e32-4309-4277-81ce-12419f0d1294
type = Encounter, id = 02ba9ec6-0712-4715-8ba4-5485fc571403
type = Questionnaire, id = argo-pl-q2
type = QuestionnaireResponse, id = argo-pl-q2-qr-06e1f0dd-5fbe-4480-9bb4-6b54ec02d31b
type = QuestionnaireResponse, id = argo-pl-q2-qr-b1cf5f57-b061-4b7f-aa9d-6283a121694b
type = QuestionnaireResponse,

WARNING,"[Bundle.entry[0].resource.ofType(Patient).identifier[2].type, Line 92, Col 26]","None of the codes provided are in the value set http://hl7.org/fhir/ValueSet/identifier-type (http://hl7.org/fhir/ValueSet/identifier-type), and a code should come from this value set unless it has no suitable code and the validator cannot judge what is suitable) (codes = http://terminology.hl7.org/CodeSystem/v2-0203#SS)"
INFORMATION,"[Bundle.entry[0].resource.ofType(Patient).extension[2], Line 54, Col 12]",This element does not match any known slice defined in the profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
INFORMATION,"[Bundle.entry[0].resource.ofType(Patient).extension[4], Line 62, Col 12]",This element does not match any known slice defined in the profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
WARNING,"[Bundle.entry[0].resource.ofType(Patient).communication[0].language, Line 180, Col 26]","None of the codes provided are in the value set http://hl7.org/fhir/us/core/ValueSet/simple-language (http://hl7.org/fhir/us/core/ValueSet/simple-language), and a code should come from this value set unless it has no suitable code and the validator cannot judge what is suitable) (codes = urn:ietf:bcp:47#en-US)"
WARNING,"[Bundle.entry[1].resource.ofType(Coverage), Line 207, Col 25]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Bundle.entry[2].resource.ofType(Organization), Line 225, Col 40]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Bundle.entry[3].resource.ofType(Location), Line 289, Col 34]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Bundle.entry[4].resource.ofType(Organization), Line 304, Col 18]",dom-6: A resource should have narrative for robust management [text.div.exists()]
INFORMATION,"[Bundle.entry[4].resource.ofType(Organization).identifier[0], Line 310, Col 12]",This element does not match any known slice defined in the profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization
WARNING,"[Bundle.entry[5].resource.ofType(Practitioner), Line 357, Col 18]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Bundle.entry[6].resource.ofType(Patient).identifier[2].type, Line 498, Col 26]","None of the codes provided are in the value set http://hl7.org/fhir/ValueSet/identifier-type (http://hl7.org/fhir/ValueSet/identifier-type), and a code should come from this value set unless it has no suitable code and the validator cannot judge what is suitable) (codes = http://terminology.hl7.org/CodeSystem/v2-0203#SS)"


### Write to File

In [8]:
if not console_only:
    file_name = f'argo-pl-{pl_use_case}-{my_b.id}.json'

    my_path = Path.cwd()/ 'test_out' / file_name
    display(HTML(f'&#9203...writing bundle resource as json file  to {my_path}... '))
    my_path.write_text(to_json(my_b))


    display(HTML(f'&#8987; Done!!!'))

### Post to Test Server

In [24]:
if not console_only:
    r = load_transaction(my_b)
    try:
        display(HTML(
            '<h1>Post Response</h1>'
            f'<h3>Status Code = {r.status_code}</h3><br />'
            f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
            f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
            '===============================================<br /><br /><br />'
            ))
    except KeyError:
        display(HTML(
            '<h1>Post Response</h1>'
            f'<h3>Status Code = {r.status_code}</h3><br />'
            f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
            f'<pre style="color: blue; background-color: Aquamarine;">Resource Body: {to_yaml(r.json())}</pre>'
            '===============================================<br /><br /><br />'
            ))   